Датасет: Womens Clothing E-Commerce Reviews.csv Для каждого уникального значения в столбце Division Name найти топ-5 самых часто используемых слов в описании отзыва. Исключить из рассмотрения стоп-слова.

In [1]:
import re
import pandas as pd
from nltk import word_tokenize
import multiprocessing as mp

In [2]:
reviews = pd.read_csv('./data/Womens Clothing E-Commerce Reviews.csv', index_col=0)

In [3]:
reviews.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
pattern = re.compile(r'[\W]+$')
reviews['tokenized'] = reviews.apply(lambda row: [word for word in word_tokenize(row['Review Text']) if not pattern.match(word)]  if not pd.isna(row['Review Text']) else [], axis=1)
reviews

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokenized
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,"[Absolutely, wonderful, silky, and, sexy, and,..."
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,"[Love, this, dress, it, 's, sooo, pretty, i, h..."
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"[I, had, such, high, hopes, for, this, dress, ..."
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"[I, love, love, love, this, jumpsuit, it, 's, ..."
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"[This, shirt, is, very, flattering, to, all, d..."
...,...,...,...,...,...,...,...,...,...,...,...
23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,"[I, was, very, happy, to, snag, this, dress, a..."
23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits,"[It, reminds, me, of, maternity, clothes, soft..."
23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses,"[This, fit, well, but, the, top, was, very, se..."
23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses,"[I, bought, this, dress, for, a, wedding, i, h..."


In [5]:
agg_reviews = reviews.groupby('Division Name')['tokenized'].agg(list)

In [6]:
%%file five_common.py
from collections import Counter
from nltk.corpus import stopwords
from nltk import flatten
def five_common(data):
    line = flatten(data)
    line = list(filter(lambda x: x not in stopwords.words('english'), line))
    counter = Counter(line)
    return (counter.most_common(5))

Overwriting five_common.py


In [7]:
from five_common import five_common

In [8]:
if __name__ == "__main__":
    with mp.Pool(processes=len(agg_reviews.values)) as pool:
        counters = pool.map(five_common, agg_reviews.values)
    

In [9]:
list(zip(agg_reviews.keys(), counters))

[('General',
  [('dress', 5999), ('I', 5552), ('size', 5151), ("n't", 4866), ("'s", 4688)]),
 ('General Petite',
  [('dress', 4260),
   ('I', 3280),
   ('size', 3034),
   ("n't", 2845),
   ('love', 2767)]),
 ('Initmates',
  [('I', 550), ('size', 504), ("n't", 473), ('love', 459), ('like', 414)])]